In [112]:
from PIL import Image
from zipfile import ZipFile
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# 

# Loading Images from Disk

In [93]:
!rm -rf ../data/good_data
!rm ../data/good_data

rm: ../data/good_data: No such file or directory


In [94]:
data_path = Path('../data/')
data_zips = [data_path / f for f in os.listdir(data_path)]

for i, f in enumerate(data_zips):
    print(f'{i}: {f}')

0: ../data/good_data.zip
1: ../data/dataCounterClockWiseMessy.zip


In [95]:
f_index = 0
data_file = data_zips[0]
print(data_file)

zip_folder_name = 'good_data'
data_extract_location = data_path / zip_folder_name

../data/good_data.zip


In [96]:
with ZipFile(data_file) as z:
    z.extractall(data_extract_location)

In [97]:
inner_folder = os.listdir(data_extract_location)[0];
data_folder_unzip = data_extract_location / inner_folder
os.listdir(data_folder_unzip)

['labels.csv', 'images']

In [98]:
labels = pd.read_csv(data_folder_unzip / 'labels.csv')
labels

,timestamp,forward/backward,left/right
0,1636315836071,0,0
1,1636315836146,0,0
2,1636315836244,0,0
3,1636315836290,0,0
4,1636315836336,0,0
...,...,...,...
9461,1636316210285,0,0
9462,1636316210333,0,0
9463,1636316210368,0,0
9464,1636316210397,0,0


In [99]:
label_encoded = labels['left/right'].values
encoder = LabelBinarizer()
encoder.fit([-1, 0, 1])
label_encoded = encoder.transform(label_encoded)
print(label_encoded[labels['left/right'] == -1])
print(label_encoded[labels['left/right'] == 0])
print(label_encoded[labels['left/right'] == 1])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]
[[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [100]:
import tensorflow.keras.preprocessing as preprocessing
image_rescale_size = (80, 60)
batch_size = 128
data_path = data_folder_unzip
print(f'{data_path}')


train_ds = preprocessing.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_rescale_size,
    shuffle=False,
    validation_split=0.2,
    subset='training',
)

val_ds = preprocessing.image_dataset_from_directory(
    str(data_path),
    labels=list(label_encoded),
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_rescale_size,
    shuffle=False,
    validation_split=0.2,
    subset='validation',
)

../data/good_data/data6
Found 9466 files belonging to 1 classes.
Using 7573 files for training.
Found 9466 files belonging to 1 classes.
Using 1893 files for validation.


In [127]:
for i, el in enumerate(train_ds):
    # print(data.shape, labels.shape)
    # print(cur_labels[:2])
    print(el[0].shape, el[1].shape)
    if i == 5:
        break


(128, 80, 60, 3) (128, 3)
(128, 80, 60, 3) (128, 3)
(128, 80, 60, 3) (128, 3)
(128, 80, 60, 3) (128, 3)
(128, 80, 60, 3) (128, 3)
(128, 80, 60, 3) (128, 3)


# Preprocessing 

## Filter labels and images where the car was moving

In [102]:
# filtered_index = (labels['forward/backward'] != 0)
# np_filter = np.array([i for i, index in enumerate(filtered_index) if index == False])
# bad_indices = tf.convert_to_tensor(np_filter); bad_indices.shape

In [103]:
# def func(index):
#     print(index)
#     return tf.reduce_any(tf.equal(index, bad_indices))
#train_ds = train_ds.enumerate()
# test = train_ds.filter(lambda i, _: func(i))

In [104]:
# ind = []
# for i, el in test.as_numpy_iterator():
#     # print(data.shape, labels.shape)
#     # print(cur_labels[:2])
#     print(el)
#     # ind.append(i)
# ind = np.array(ind)

# Dense Neural Network


In [105]:
model = Sequential()
model.add(Flatten())
model.add(Rescaling(scale=1./255))
model.add(Dense(32))
model.add(Dense(64))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(3, activation='softmax'))

In [109]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
#model.summary()

In [110]:
model.fit(train_ds, validation_data=val_ds, epochs=50)

Epoch 1/50
60/60 [==============================] - 11s 182ms/step - loss: 1359.1945 - accuracy: 0.5899 - val_loss: 333.9830 - val_accuracy: 0.6667
Epoch 2/50
60/60 [==============================] - 12s 197ms/step - loss: 386.9763 - accuracy: 0.6171 - val_loss: 222.1673 - val_accuracy: 0.6080
Epoch 3/50
60/60 [==============================] - 12s 195ms/step - loss: 133.2527 - accuracy: 0.6033 - val_loss: 99.5995 - val_accuracy: 0.5193
Epoch 4/50
60/60 [==============================] - 10s 173ms/step - loss: 136.5300 - accuracy: 0.6279 - val_loss: 75.7496 - val_accuracy: 0.6883
Epoch 5/50
60/60 [==============================] - 10s 173ms/step - loss: 92.1265 - accuracy: 0.6242 - val_loss: 71.0108 - val_accuracy: 0.7031
Epoch 6/50
60/60 [==============================] - 10s 165ms/step - loss: 142.1997 - accuracy: 0.5590 - val_loss: 53.0895 - val_accuracy: 0.7137
Epoch 7/50
60/60 [==============================] - 10s 168ms/step - loss: 55.5201 - accuracy: 0.6228 - val_loss: 184.6105

KeyboardInterrupt: 

# CNN

In [123]:
model = Sequential()
model.add(Rescaling(scale=1./255))
model.add(Conv2D(32, (3,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32, (3,3)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(3, activation='softmax'))

In [128]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [129]:
model.fit(train_ds, validation_data=val_ds, epochs=50)

Epoch 1/50
60/60 [==============================] - 26s 425ms/step - loss: 213.4039 - accuracy: 0.4754 - val_loss: 4.8985 - val_accuracy: 0.3080
Epoch 2/50
60/60 [==============================] - 26s 427ms/step - loss: 4.8267 - accuracy: 0.5740 - val_loss: 0.8960 - val_accuracy: 0.6123
Epoch 3/50
60/60 [==============================] - 29s 477ms/step - loss: 0.9280 - accuracy: 0.6520 - val_loss: 0.7789 - val_accuracy: 0.6889
Epoch 4/50
60/60 [==============================] - 24s 407ms/step - loss: 0.7541 - accuracy: 0.7263 - val_loss: 0.7695 - val_accuracy: 0.6809
Epoch 5/50
60/60 [==============================] - 24s 404ms/step - loss: 0.7310 - accuracy: 0.7262 - val_loss: 0.7591 - val_accuracy: 0.6904
Epoch 6/50
60/60 [==============================] - 24s 402ms/step - loss: 0.7123 - accuracy: 0.7218 - val_loss: 0.7519 - val_accuracy: 0.6910
Epoch 7/50
60/60 [==============================] - 26s 440ms/step - loss: 0.6988 - accuracy: 0.7296 - val_loss: 0.7292 - val_accuracy: 0.69

KeyboardInterrupt: 